In [1]:
import pandas as pd
import datetime as dt
from modules import crfb
from modules import covid

ModuleNotFoundError: No module named 'openpyxl'

In [17]:
# Get the most recent dataset that we have on file
crfb_df = crfb.get_data_single()
# Download all the datasets we have, using the column 'Date CRFB Downloaded'
#crfb_df = crfb.get_data_all()
crfb_df

loading data from date 2021-02-09 : file 20210209_cmt.xlsx...
failed to load data C:\Users\st1kinneia\Desktop\Control-Alt-Elite\modules\crfb_raw_data\20210209_cmt.xlsx
ERROR: Your version of xlrd is 2.0.1. In xlrd >= 2.0, only the xls format is supported. Install openpyxl instead.


""


In [8]:
# To filter by date use
#covid_df = covid.get_data(date_min=dt.datetime(2020, 10, 23), date_max=dt.datetime(2021, 2, 9))
# else, use this
covid_df = covid.get_data()
covid_df

,date,state,positive,negative,onVentilatorCumulative,recovered,death,hospitalized,positiveTests
1,2021-02-09,Alabama,474666.0,1825185.0,1463.0,252880.0,8579.0,43499.0,0
2,2021-02-09,Arkansas,308848.0,2297201.0,1464.0,288774.0,5148.0,14174.0,74670
18,2021-02-09,Kansas,282960.0,920799.0,411.0,5954.0,4197.0,8749.0,0
48,2021-02-09,Utah,356040.0,1467542.0,754.0,324102.0,1748.0,13950.0,436658
57,2021-02-08,Alabama,473348.0,1820080.0,1460.0,252880.0,8523.0,43383.0,0
...,...,...,...,...,...,...,...,...,...
17362,2020-04-05,Arkansas,830.0,10412.0,39.0,97.0,16.0,130.0,0
17418,2020-04-04,Arkansas,743.0,9627.0,39.0,79.0,14.0,106.0,0
17474,2020-04-03,Arkansas,704.0,8995.0,39.0,60.0,12.0,105.0,0
17530,2020-04-02,Arkansas,643.0,7880.0,32.0,47.0,12.0,100.0,0


In [11]:
state_leg = crfb_df.groupby('Recipient State')['Legislation'].count().sort_values(ascending=False).to_frame()
state_leg.reset_index(inplace=True)

KeyError: 'Recipient State'

In [ ]:
state_money = crfb_df.groupby('Recipient State')['Amount Committed/Disbursed'].sum().sort_values(ascending=False).to_frame()
state_money.reset_index(inplace=True)
state_money['Amount Committed/Disbursed'] = state_money.apply(lambda x: "${0:,.2f}".format(x['Amount Committed/Disbursed']), axis=1)
state_money.rename(columns={"Recipient State": "state"}, inplace=True)

In [ ]:
state_death = covid_df.groupby('state')['death'].max().to_frame()
state_death.reset_index(inplace=True)

In [ ]:
state_cases = covid_df.groupby('state')['positive'].max().to_frame()
state_cases.reset_index(inplace=True)

In [ ]:
sample_state = pd.merge(state_cases,state_death,on='state')
sample_state['Death %'] = round(sample_state['death']/sample_state['positive'] * 100,2)

In [ ]:
sample_state = pd.merge(sample_state,state_money,on='state',how='left')
sample_state

In [ ]:
## day_change
# day change per state per date. date, year, month, day, +positive +death +recovered +money recieved
## week_change
# week change per state, same values
## month_change
# month change per state, same values